[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/HandRefiner-colab/blob/main/HandRefiner_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/HandRefiner
%cd /content/HandRefiner

!pip install -q pytorch-lightning omegaconf einops yacs trimesh mediapipe rtree boto3 gradio==3.50.2
!pip install -q git+https://gitlab.eecs.umich.edu/ngv-python-modules/opendr
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!apt -y install -qq aria2

%cd /content/HandRefiner
!git clone -b dev --recursive https://github.com/camenduru/MeshGraphormer
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl -d /content/HandRefiner/MeshGraphormer/src/modeling/data -o basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/MANO_RIGHT.pkl -d /content/HandRefiner/MeshGraphormer/src/modeling/data -o MANO_RIGHT.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/graphormer_release/graphormer_3dpw_state_dict.bin -d /content/HandRefiner/MeshGraphormer/models/graphormer_release -o graphormer_3dpw_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/graphormer_release/graphormer_h36m_state_dict.bin -d /content/HandRefiner/MeshGraphormer/models/graphormer_release -o graphormer_h36m_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/graphormer_release/graphormer_hand_state_dict.bin -d /content/HandRefiner/MeshGraphormer/models/graphormer_release -o graphormer_hand_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/raw/main/hrnet/cls_hrnet_w40_sgd_lr5e-2_wd1e-4_bs32_x100.yaml -d /content/HandRefiner/MeshGraphormer/models/hrnet -o cls_hrnet_w40_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/raw/main/hrnet/cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml -d /content/HandRefiner/MeshGraphormer/models/hrnet -o cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/hrnet/hrnetv2_w40_imagenet_pretrained.pth -d /content/HandRefiner/MeshGraphormer/models/hrnet -o hrnetv2_w40_imagenet_pretrained.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/hrnet/hrnetv2_w64_imagenet_pretrained.pth -d /content/HandRefiner/MeshGraphormer/models/hrnet -o hrnetv2_w64_imagenet_pretrained.pth
%cd /content/HandRefiner/MeshGraphormer
!pip install -e .
!pip install -e ./manopth/.

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/hand_landmarker.task -d /content/HandRefiner/preprocessor -o hand_landmarker.task
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/inpaint_depth_control.ckpt -d /content/HandRefiner/models -o inpaint_depth_control.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/ckpt200-multisc-pred.zip -d /content/HandRefiner/MeshGraphormer/predictions -o ckpt200-multisc-pred.zip
!mkdir /content/HandRefiner/output
%cd /content/HandRefiner

In [ ]:
%cd /content/HandRefiner

from __future__ import absolute_import, division, print_function
import sys
from config import handrefiner_root
import os

def load():
    paths = [handrefiner_root, os.path.join(handrefiner_root, 'MeshGraphormer'), os.path.join(handrefiner_root, 'preprocessor')]
    for p in paths:
        sys.path.insert(0, p)

load()

import argparse
import json
import torch
import numpy as np
import cv2

from PIL import Image
from torchvision import transforms
import numpy as np
import cv2

from pytorch_lightning import seed_everything
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler
import config

import cv2
import einops
import numpy as np
import torch
import random
from pathlib import Path
from preprocessor.meshgraphormer import MeshGraphormerMediapipe
import ast

transform = transforms.Compose([           
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])])

model = create_model("control_depth_inpaint.yaml").cpu()
model.load_state_dict(load_state_dict('/content/HandRefiner/models/inpaint_depth_control.ckpt', location='cuda'), strict=False)
model = model.to("cuda")

In [ ]:
def inference(image_path, prompt, seed):
    meshgraphormer = MeshGraphormerMediapipe()
    image = np.array(Image.open(image_path))
    raw_image = image
    H, W, C = raw_image.shape
    gen_count = 0

    file_name_raw = Path(image_path).stem
    depthmap, mask, info = meshgraphormer.get_depth(os.path.dirname(image_path), os.path.basename(image_path), 30)
    cv2.imwrite("img_depth.jpg", depthmap)
    cv2.imwrite("img_mask.jpg", mask)
    control = depthmap

    ddim_sampler = DDIMSampler(model)
    out_dir="/content/HandRefiner/output"
    num_samples = 1
    ddim_steps = 50
    guess_mode = False
    adaptive_control = False
    eval=False
    strength = 1.0
    scale = 9.0

    a_prompt = "realistic, best quality, extremely detailed"
    n_prompt = "fake 3D rendered image, longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, blue"

    source = raw_image

    source = (source.astype(np.float32) / 127.5) - 1.0
    source = source.transpose([2, 0, 1])  # source is c h w

    mask = mask.astype(np.float32) / 255.0
    mask = mask[None]
    mask[mask < 0.5] = 0
    mask[mask >= 0.5] = 1

    hint = control.astype(np.float32) / 255.0

    masked_image = source * (mask < 0.5)  # masked image is c h w

    mask = torch.stack([torch.tensor(mask) for _ in range(num_samples)], dim=0).to("cuda")
    mask = torch.nn.functional.interpolate(mask, size=(64, 64))

    if seed == -1:
        seed = random.randint(0, 65535)
    seed_everything(seed)

    if config.save_memory:
        model.low_vram_shift(is_diffusing=False)

    masked_image = torch.stack(
        [torch.tensor(masked_image) for _ in range(num_samples)], dim=0
    ).to("cuda")

    # this should be b,c,h,w
    masked_image = model.get_first_stage_encoding(model.encode_first_stage(masked_image))

    x = torch.stack([torch.tensor(source) for _ in range(num_samples)], dim=0).to("cuda")
    z = model.get_first_stage_encoding(model.encode_first_stage(x))

    cats = torch.cat([mask, masked_image], dim=1)

    hint = hint[
        None,
    ].repeat(3, axis=0)

    hint = torch.stack([torch.tensor(hint) for _ in range(num_samples)], dim=0).to("cuda")

    cond = {
        "c_concat": [cats],
        "c_control": [hint],
        "c_crossattn": [model.get_learned_conditioning([prompt] * num_samples)],
    }
    un_cond = {
        "c_concat": [cats],
        "c_control": [hint],
        "c_crossattn": [model.get_learned_conditioning([n_prompt] * num_samples)],
    }


    shape = (4, H // 8, W // 8)

    if config.save_memory:
        model.low_vram_shift(is_diffusing=True)

    if not adaptive_control:
        seed_everything(seed)
        model.control_scales = (
            [strength * (0.825 ** float(12 - i)) for i in range(13)]
            if guess_mode
            else ([strength] * 13)
        )  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
        samples, intermediates = ddim_sampler.sample(
            ddim_steps,
            num_samples,
            shape,
            cond,
            verbose=False,
            unconditional_guidance_scale=scale,
            unconditional_conditioning=un_cond,
            x0=z,
            mask=mask
        )
        if config.save_memory:
            model.low_vram_shift(is_diffusing=False)

        x_samples = model.decode_first_stage(samples)
        # print(x_samples.shape)
        x_samples = (
            (einops.rearrange(x_samples, "b c h w -> b h w c") * 127.5 + 127.5)
            .cpu()
            .numpy()
            .clip(0, 255)
            .astype(np.uint8)
        )

        if eval: # currently only works for batch size of 1 
            assert num_samples == 1, "MPJPE evaluation currently only works for batch size of 1"
            mpjpe = meshgraphormer.eval_mpjpe(x_samples[0], info)
            print(mpjpe)
        for i in range(num_samples):
            cv2.imwrite('/content/HandRefiner/output/output_image.jpg', cv2.cvtColor(x_samples[i], cv2.COLOR_RGB2BGR))
            gen_count += 1
            return '/content/HandRefiner/output/output_image.jpg'
    else:
        assert num_samples == 1, "Adaptive thresholding currently only works for batch size of 1"
        strengths = [1.0, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        ref_mpjpe = None
        chosen_strength = None
        final_mpjpe = None
        chosen_sample = None
        count = 0
        for strength in strengths:
            seed_everything(seed)
            model.control_scales = (
                [strength * (0.825 ** float(12 - i)) for i in range(13)]
                if guess_mode
                else ([strength] * 13)
            )  # Magic number. IDK why. Perhaps because 0.825**12<0.01 but 0.826**12>0.01
            samples, intermediates = ddim_sampler.sample(
                ddim_steps,
                num_samples,
                shape,
                cond,
                verbose=False,
                unconditional_guidance_scale=scale,
                unconditional_conditioning=un_cond,
                x0=z,
                mask=mask
            )
            if config.save_memory:
                model.low_vram_shift(is_diffusing=False)

            x_samples = model.decode_first_stage(samples)

            x_samples = (
                (einops.rearrange(x_samples, "b c h w -> b h w c") * 127.5 + 127.5)
                .cpu()
                .numpy()
                .clip(0, 255)
                .astype(np.uint8)
            )
            mpjpe = meshgraphormer.eval_mpjpe(x_samples[0], info)
            if count == 0:
                ref_mpjpe = mpjpe
                chosen_sample = x_samples[0]
            elif mpjpe < ref_mpjpe * 1.15:
                chosen_strength = strength
                final_mpjpe = mpjpe
                chosen_sample = x_samples[0]
                break
            elif strength == 0.9:
                final_mpjpe = ref_mpjpe
                chosen_strength = 1.0
            count += 1
        
        cv2.imwrite('/content/HandRefiner/output/output_image.jpg', cv2.cvtColor(x_samples[0], cv2.COLOR_RGB2BGR))
        gen_count += 1
        return '/content/HandRefiner/output/output_image.jpg'

In [ ]:
output_image = inference('/content/examples_IMG_1050.jpg', "a cute rabbit, white background, pastel hues, minimal illustration, line art, pen drawing", 1)
output_image